<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
with open(f'./module1-rnn-and-lstm/100-0.txt', 'r') as f:
    text = f.read()

In [2]:
chars = list(set(text))

char_indices = {c:i for i,c in enumerate(chars)}
indices_char = {i:c for i,c in enumerate(chars)}
char_indices[''] = max(char_indices.values()) + 1
indices_char[max(char_indices.values()) + 1] = ''
char_indices

{'#': 0,
 's': 1,
 'u': 2,
 '(': 3,
 '”': 4,
 'É': 5,
 'o': 6,
 '.': 7,
 '3': 8,
 'æ': 9,
 ' ': 10,
 'O': 11,
 'Æ': 12,
 'g': 13,
 'n': 14,
 'I': 15,
 '9': 16,
 'N': 17,
 'ê': 18,
 ']': 19,
 'A': 20,
 '$': 21,
 'h': 22,
 'B': 23,
 'y': 24,
 'd': 25,
 'U': 26,
 "'": 27,
 '}': 28,
 'J': 29,
 'S': 30,
 'G': 31,
 'é': 32,
 'è': 33,
 'a': 34,
 'm': 35,
 'œ': 36,
 'î': 37,
 '1': 38,
 'W': 39,
 '"': 40,
 't': 41,
 'v': 42,
 'L': 43,
 'M': 44,
 'P': 45,
 'f': 46,
 '\ufeff': 47,
 'à': 48,
 '6': 49,
 ',': 50,
 ';': 51,
 'q': 52,
 'r': 53,
 'C': 54,
 'b': 55,
 'k': 56,
 '[': 57,
 'Z': 58,
 'Y': 59,
 '0': 60,
 '_': 61,
 '`': 62,
 '5': 63,
 '|': 64,
 'V': 65,
 'p': 66,
 '\\': 67,
 'i': 68,
 'c': 69,
 'E': 70,
 ':': 71,
 '@': 72,
 'e': 73,
 'j': 74,
 '4': 75,
 'D': 76,
 'K': 77,
 '?': 78,
 '/': 79,
 '\t': 80,
 '‘': 81,
 'X': 82,
 'F': 83,
 'H': 84,
 'w': 85,
 '!': 86,
 '7': 87,
 'l': 88,
 '%': 89,
 'x': 90,
 '*': 91,
 '—': 92,
 'T': 93,
 '\n': 94,
 'Q': 95,
 '&': 96,
 '8': 97,
 'â': 98,
 '-': 99,
 '

In [3]:
max_len = 71

In [4]:
# text_sequences = text.split('\n')
text_sequences = []
step = 40
for i in range(0, len(text) - max_len, step):
    text_sequences.append(text[i: i + max_len])
text_sequences = [s for s in text_sequences if len(s) > 0 and len(s) <= max_len]
text_sequences[1206]

'd should look into your moan,\n  And mock you with me after I am gone.\n\n'

In [5]:
len(text_sequences)

139328

In [6]:
char_sequences = [
    [char_indices[c] for c in text_sequence]
        for text_sequence in text_sequences
]
for char_sequence in char_sequences:
    while len(char_sequence) < max_len:
        char_sequence.append(char_indices[''])
char_sequences[1206]

[25,
 10,
 1,
 22,
 6,
 2,
 88,
 25,
 10,
 88,
 6,
 6,
 56,
 10,
 68,
 14,
 41,
 6,
 10,
 24,
 6,
 2,
 53,
 10,
 35,
 6,
 34,
 14,
 50,
 94,
 10,
 10,
 20,
 14,
 25,
 10,
 35,
 6,
 69,
 56,
 10,
 24,
 6,
 2,
 10,
 85,
 68,
 41,
 22,
 10,
 35,
 73,
 10,
 34,
 46,
 41,
 73,
 53,
 10,
 15,
 10,
 34,
 35,
 10,
 13,
 6,
 14,
 73,
 7,
 94,
 94]

In [7]:
indices_char[64]

'|'

In [8]:
import numpy as np

x = np.zeros((len(char_sequences), max_len, len(char_indices)), dtype=np.bool)
y = np.zeros((len(char_sequences), len(char_indices)), dtype=np.bool)

for i, sequence in enumerate(char_sequences):
    if i == len(char_sequences) - 1: break

    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, char_sequences[i+1][0]] = 1

In [9]:
x[1][0]

array([False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [10]:
y[0]

array([False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [11]:
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(LSTM(128,input_shape=(max_len, len(char_indices))))
model.add(Dense(len(char_indices), activation='softmax'))

          
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [12]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
from tensorflow.keras.callbacks import LambdaCallback
import random
import sys

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - max_len - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + max_len]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        # sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, max_len, len(char_indices)))
            for t, char in enumerate(sentence[-max_len:]):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence += next_char

            if i%40 == 0:
                sys.stdout.write(str(i//40))
                sys.stdout.flush()
        print(sentence + '\n')

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [14]:
model.fit(x, y,
          batch_size=512,
          epochs=8,
          callbacks=[print_callback])

Train on 139328 samples
Epoch 1/5
139264/139328 [============================>.] - ETA: 0s - loss: 3.2935
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "panion!
                            [The PROVOST lays bands on the DUKE"
0123456789panion!
                            [The PROVOST lays bands on the DUKE                                                                                                                            n                  s                                                                                                                                      e                                                                                                 e                       

----- diversity: 0.5
----- Generating with seed: "panion!
                            [The PROVOST lays bands on the DUKE"
0123456789panion!
                            [The PROVOST lays bands on the DUKE n    s          sa              o  t i ie     

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN